# RAG 



## Dados - Livros

download books: https://drive.google.com/drive/folders/1yhocUZ1cGfZiBg63hBlH8mXe2oHXYRuT?usp=sharing

Estrutura:
```
books-rag/
├── raw/                         # originais (NÃO edite)
│   ├── pdf/
│   └── epub/
├── working/                     # intermediários
│   ├── converted/               # md/txt por capítulo ou seção
│   ├── ocr/                     # PDFs escaneados corrigidos
│   └── manifests/               # CSV/JSON com inventário e status
├── corpus/                      # texto limpo + chunks com metadados
│   ├── jsonl/                   # um chunk por linha
│   └── parquet/                 # (opcional) rápido p/ análise
├── index/                       # índices de busca
│   ├── faiss/
│   └── bm25/
└── evaluation/                  # perguntas, gabaritos, logs

### Extract Book Data

In [13]:
!ls

Books-20251007T165548Z-1-001	  books_rag	     README.md
Books-20251007T165548Z-1-001.zip  experiments.ipynb  src


In [ ]:
!python src/extract_book_data.py \
  --root "/home/pdi_4/Documents/Documentos/rag/books_rag" \
  --pdf_dir "/home/pdi_4/Documents/Documentos/rag/books_rag/raw/pdf" \
  --epub_dir "/home/pdi_4/Documents/Documentos/rag/books_rag/raw/epub"


Encontrados: 18 PDF(s) em /home/pdi_4/Documents/Documentos/rag/books_rag/raw/pdf
Encontrados: 6 EPUB(s) em /home/pdi_4/Documents/Documentos/rag/books_rag/raw/epub
Chunking PDF Dorothy R. Kirk - Deciphering Object-Oriented Programming with C++ 


## Tipo de RAG